In [38]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [39]:
import math
"""
def detectYW(image): #Detects yellow and white color in image
    
    #Detecting yellow and white in the image
    imgHSV=cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # lower mask
    lower_yellow = np.array([40,0,200])
    upper_yellow = np.array([100,255,255])
    mask0 = cv2.inRange(imgHSV, lower_yellow, upper_yellow)

    # upper mask
    lower_white = np.array([0,0,215])
    upper_white = np.array([255,80,255])
    mask1 = cv2.inRange(imgHSV, lower_white, upper_white)

    # join my masks
    mask = mask0+mask1

    #Color image
    colorImg = image.copy()
    colorImg[np.where(mask==0)] = 0
    
    return colorImg
"""
def grayscale(img):
    #Applies the Grayscale transform
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
# Lists for lines
    ysize, xsize, _ = img.shape
    linePos = []
    lineNeg = []
    
    #Counters
    i_p = 0
    i_n = 0
    cutoff = 0.55


    for line in lines:
        print(line)
        for x1,y1,x2,y2 in line:
        '''    
            m = ((y2-y1)/(x2-x1))
            print(m)
            print(x1,',', xsize/2)
            
            if m < 0:
                lineNeg += [((x1+x2)/2, (y1+y2)/2)]
            elif m > 0:
                linePos += [((x1+x2)/2, (y1+y2)/2)]
        '''
        if x1 > xsize/2:
            lineNeg += [[x1,y1], [x2,y2]]
        elif x1 < xsize/2:
            linePos += [[x1,y1], [x2,y2]]
    

    lineNegS = sorted(lineNeg , key = lambda x: x[1], reverse = True)
    linePosS = sorted(linePos , key = lambda x: x[1], reverse = True)
    lineNeg1 = np.array(lineNeg)
    linePos1 = np.array(linePos)
    '''
    mNeg, rightB = np.polyfit(lineNeg1[:,0], lineNeg1[:,1], 1)
    mPos, leftB = np.polyfit(linePos1[:,0], linePos1[:,1],1)

    
    print('---------------')
    print(mNeg, ',', rightB)
    print(mPos, ',', leftB)
    print('---------------')
    
    print(len(linePos1))
    print(lineNeg1)
    print('------')
    print(linePos1)
    print('------')
    print(lineNegS)
    print('------')
    print(linePosS)
        
    
    cutoff = int(0.65*ysize)
    
    print(cutoff)
    
    left_line = (int((cutoff-leftB)/mPos), cutoff, int((ysize-leftB)/mPos), ysize)
    print(left_line)

    right_line = (int((cutoff-rightB)/mNeg), cutoff, int((ysize-rightB)/mNeg), ysize)
    print(right_line)

    # Draw the predicted lane lines on the image.
    for x1,y1,x2,y2 in (left_line, right_line):
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    '''    
    
    mNeg, right_b = np.polyfit(lineNeg[:,0], lineNeg[:,1], 1)
    
    print(mNeg, ',', right_b)
    
    mPos, left_b  = np.polyfit(linePos[:,0], linePos[:,1], 1) 
    
    print(mPos, ',', left_b)

    cutoff = int(0.65*ysize)
    
    print(cutoff)
    
    left_line = (int((cutoff-left_b)/mPos), cutoff, int((ysize-left_b)/mPos), ysize)
    print(left_line)

    right_line = (int((cutoff-right_b)/mNeg), cutoff, int((ysize-right_b)/mNeg), ysize)
    print(right_line)

    # Draw the predicted lane lines on the image.
    for x1,y1,x2,y2 in (left_line, right_line):
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        
        
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return lines,line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):

    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image): #Lane lines detection pipeline

    imshape = image.shape
    pXmax=imshape[1]
    pYmax=imshape[0]
    
    #Defining ROI vertices and selecting ROI in image
    yMin=pYmax/2+35
    yMax=pYmax
    roi=[(100,yMax),(pXmax/2-15, yMin), (pXmax/2+25, yMin), (pXmax-80,yMax)]
    vertices = np.array([roi], dtype=np.int32)
    Img=region_of_interest(image, vertices)
    
    #Detecting yellow and white color in image
    #Img=detectYW(Img)
    
    #Converting image to grayscale
    Img = grayscale(Img)
    
    #Applying Gaussian smoothing
    kernel_size = 11
    Img=gaussian_blur(Img, kernel_size)
    
    #Applying Canny edge detection
    low_threshold = 40
    high_threshold = 50
    Img = canny(Img, low_threshold, high_threshold)
    plt.imshow(Img)

    #Applying Hough transformation
    rho=1 #distance resolution in pixels of the Hough grid
    theta=np.pi/180 #angular resolution in radians of the Hough grid
    threshold=40 #minimum number of votes (intersections in Hough grid cell)
    min_line_len = 30 #minimum number of pixels making up a line
    max_line_gap = 200 #maximum gap in pixels between connectable line segments
    lines, Img=hough_lines(Img, rho, theta, threshold, min_line_len, max_line_gap)
    
    #Drawing lanes lines on the image
    #pImg=draw_lines(lines, image, yMin, yMax)     
    Img=weighted_img(Img, image, α=0.8, β=1.0, λ=0.)
    
    return Img


IndentationError: expected an indented block (<ipython-input-39-eb6895312eb6>, line 99)

In [ ]:
import os
imgs=os.listdir("test_images/")
print(imgs)
image_output_dir="test_images_output/"

for i, fig in enumerate(imgs):
    #Reading in an image
    image = mpimg.imread("test_images/"+fig)
    #Processing image
    print(fig)
    pImg=process_image(image);
    plot=plt.imshow(pImg)
    plt.savefig(image_output_dir+fig, format='jpg', dpi=300)

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))